# Converting geopackage to Earth Engine object

We will see how to convert a specific layer from geopackage file to an EE object. Currently the geemap module does not have support for this format (.gpkg).

## Importing used libraries

In [ ]:
import ee # # Authenticates and initializes Earth Engine
import geemap # A dynamic module of Google Earth Engine
import fiona # For geopackage file handling 
import os #To set and modify filepaths
import json # For json objects
import urllib # For download geopackage files

## Geopackage data

In this case, our input will be a URL, though can be a filepath

In [ ]:
#A sample of ~6k restaurants in Metropolita Lima
url_gpkg= "https://github.com/jfphlorecti/GEE/raw/master/data/restaurants.gpkg"

## Defining functions

In [ ]:
def ee_initialize(): ## Initializing in Google Earth Engine as function
    try:
        ee.Initialize()
    except:
        ee.Authenticate()
        ee.Initialize()

This function allows us to know if the input path is a URL or a filepath

In [ ]:
def is_url(url): # URL Input verification
    """Check to see if *url* has a valid protocol."""
    import urllib 
    try:
        return urllib.parse.urlparse(url).scheme in ('http', 'https') 
    except Exception:
        return False

## NOTE
To convert a geopackage file to an ee object, you need to first convert it in a geojson file, this  provide us an exist way to obtain an ee object (with geojson_to_ee)

## Geopackage file to geojson

### Summary of Process
First, We define the ways of saving an output json file wether URL or filepath. There are three ways: if your route is defined or not, if it is duplicated and if the route does not yet exist.

Meanwhile, I  make fiona read this files in two different ways (as URL and filepath), I use fiona package to handling geopackage file, since package have supported drivers (.gpkg extension). Then, I extract similar keys of geojson (id, properties and geometry). 

These results are stored in a buffer list. Next, I open the empty output json and write in it: type: feature Collection and in features I add buffer list. Finally I load it and ready! We already have our geojson.

In [ ]:
def gpkg_to_geojson(in_gpkg, layer= None, out_json=None):
    """Converts a geopackage to GeoJSON.
    Args:
        in_gpkg (str): File path or URL of the input geopackage.
        layer (str, optional): Layer of the input geopackage. Defaults first layer as None
        out_json (str, optional): File path of the output GeoJSON. Defaults to None.
    Returns:
        object: The json object representing the geopackage.
    """   
    try: 
        import fiona
        import json  
        if os.path.exists(in_gpkg): # If the path is a filepath      
                path_or_bytes = os.path.abspath(in_gpkg)
                reader = fiona.open
                if out_json is None:  ## Obtaining empty output json
                    out_json = os.path.splitext(in_gpkg)[0] + ".json"
                elif os.path.exists(out_json): # If the out_json is duplicated
                    out_json = out_json.replace('.json', '_bk.json')  
                elif not os.path.exists(os.path.dirname(out_json)): # If the filepath has not been created yet
                    os.makedirs(os.path.dirname(out_json))    
      
                    
        elif is_url(in_gpkg): # If the path is a URL                  
                path_or_bytes = urllib.request.urlopen(in_gpkg).read()
                reader = fiona.BytesCollection            
                if out_json is None: # If the ouput name of the json is not specified
                    out_json = os.path.split(in_gpkg)[1].split(".")[0] + ".json"
                elif os.path.exists(out_json): # If the out_json is duplicated
                    out_json = out_json.replace('.json', '_bk.json')    
                elif not os.path.exists(os.path.dirname(out_json)): # If the filepath has not been created yet
                    os.makedirs(os.path.dirname(out_json))             

        buffer=[]         
        with reader(path_or_bytes, layer = layer, enabled_drivers="GPKG") as features:
            for feature in features: #Reading each feature of geopackage we obtain a dict keys for json
                    ids = feature["id"]
                    atr = feature["properties"]
                    geom = feature["geometry"]
                    buffer.append(dict(id = ids, type="Feature", properties=atr, geometry=geom)) 

        with open(out_json, "w") as geojson: 
            geojson.write(json.dumps({"type": "FeatureCollection", #Writing in a json our buffer list
                                 "features":buffer}, indent=2))
            geojson.close()          
        with open(out_json) as f:
             json_data = json.load(f) #Reading a full json and return it as result
       
        return json_data

    except Exception as e:
        print(e)

Next, after that, we have a made definition of geojson_to_ee obtained in [geemap](https://github.com/giswqs/geemap/blob/master/geemap/geemap.py)

In [ ]:
def geojson_to_ee(geo_json, geodesic=True):
    """Converts a geojson to ee.Geometry()
    Args:
        geo_json (dict): A geojson geometry dictionary or file path.
    Returns:
        ee_object: An ee.Geometry object
    """
    ee_initialize()

    try:
        import json

        if not isinstance(geo_json, dict) and os.path.isfile(geo_json):
            with open(os.path.abspath(geo_json)) as f:
                geo_json = json.load(f)

        if geo_json['type'] == 'FeatureCollection':
            features = ee.FeatureCollection(geo_json['features'])
            return features
        elif geo_json['type'] == 'Feature':
            geom = None
            keys = geo_json['properties']['style'].keys()
            if 'radius' in keys:  # Checks whether it is a circle
                geom = ee.Geometry(geo_json['geometry'])
                radius = geo_json['properties']['style']['radius']
                geom = geom.buffer(radius)
            elif geo_json['geometry']['type'] == 'Point':  # Checks whether it is a point
                coordinates = geo_json['geometry']['coordinates']
                longitude = coordinates[0]
                latitude = coordinates[1]
                geom = ee.Geometry.Point(longitude, latitude)
            else:
                geom = ee.Geometry(geo_json['geometry'], "", geodesic)
            return geom
        else:
            print( "Could not convert the geojson to ee.Geometry()" )
    except Exception as e:
            print("Could not convert the geojson to ee.Geometry()")
            print(e)

### Geopackage file in Earth Engine object

For last, I merge above functions in one. Finally we have gpkg_to_ee function

In [ ]:
def gpkg_to_ee(in_gpkg, layer=None):
    """Converts a geopackage to Earth Engine objects.
    Args:
        in_gpkg (str): File path to a geopackage.
    Returns:
        object: Earth Engine objects representing the geopackage
    """
    ee_initialize()
    try:
        json_data = gpkg_to_geojson(in_gpkg, layer= layer) ## Converting geopackage file to geojson
        ee_object  = geojson_to_ee(json_data) ## Converting geojson to ee object
        return ee_object
    except Exception as e:
        print(e)

## Testing our gkpg_to_geojson and gpkg_to_ee functions

In [ ]:
## From URL
url = "https://github.com/jfphlorecti/GEE/raw/master/data/restaurants.gpkg"
restaurants_json= gpkg_to_geojson(url, layer="restaurants")
restaurants_json

In [ ]:
## From filepath 
import os
workspace = r"C:\Users\jf_ph\OneDrive\Documentos\Python Scripts\gee\data"
geegpkg = os.path.join(workspace,"geegpkg.gpkg")
gpkg_json= gpkg_to_geojson(geegpkg,layer="departamentos")
gpkg_json

In [ ]:
## Creating a ee.FeatureCollection
restaurants_ee= gpkg_to_ee(url)
restaurants_ee

In [ ]:
##Visualizing on Google Earth Engine map
Map = geemap.Map(center =[-12.07,-77.11],zoom = 9.5)
Map.addLayer(restaurants_ee,{"color":"darkorange"},"Restaurants in Metropolitan Lima")
Map